## Procesamiento de Lenguaje Natural

*MINI-TASK \#2* 

# ***Corrector ortográfico de P. Norvig***

### **Equipo:**

- Burruel Durán Luis Andrés
-
- Villalba Miranda Jesús Abraham

**Fuentes**
* [How to Write a Spelling Corrector (Peter Norvig) ](https://norvig.com/spell-correct.html)
---

## **I. Introducción**

En el mundo en el que siempre estamos redactando información, séase mediante la generación de documentos o por la comunicación por medio de mensajes, una de las características más sobresalientes de los editores de texto que utilizamos, es la capacidad de corregir aquellas palabras que escribimos incorrectamente.

[Corrector ortográfico ](https://es.wikipedia.org/wiki/Corrector_ortogr%C3%A1fico#:~:text=Un%20corrector%20ortogr%C3%A1fico%20es%2C%20en,al%20usuario%20en%20su%20escritura.)
 > Un *corrector ortográfico* es una aplicación de software que se utiliza para analizar textos con el fin de detectar y, de forma automática o manual, corregir faltas ortográficas ayudando al usuario en su escritura.

### ¿Cómo funciona un corrector ortográfico?
Un corrector ortográfico sigue el siguiente proceso:
 1. **Identifica la palabra incorrecta**
    
    Identificamos que una palabra es incorrecta, es decir, no se encuentra dentro de nuestro vocabulario.

 2. **Se calculan las palabras a 1, 2 o 3 de distancia**
    
    Para esto se utiliza el algoritmo de **distancia mínima de edición**, lo que nos ayuda a tener una noción de similaridad entre dos palabras. 
    
    La **distancia mínima de edición** entre dos palabras la podemos definir como el mínimo número de operaciones de edición para transformar una cadena de caracteres (*source*) en otra (*target*).

    Las operaciones de edición pueden ser inserción, eliminación y remplazo de un carácter. A estas operaciones se les asigna un peso y basado en este, se obtiene la distancia.

 3. **Se filtran los posibles candidatos**
    
    De las palabras obtenidas en el paso anterior, se filtran los posibles candidatos de manera que se encuentren dentro de nuestro vocabulario.
    
 4. **Se calcula el más probable en funcion del contexto**

    Intentamos encontrar la corrección *c*, de todos los candidatos, de forma que maximize la probabilidad de que *c* es la corrección (palabra) correcta, dada la palabra incorrecta original. Para obtener dicha corrección, nos basamos en nuestro corpus.

Durante el resto del documento, explicaremos como funciona nuestra implementación de un corrector ortográfico basada en la libreta realizada por *P. Norvig*. Las secciones las podemos dividir en: II. ¿Cómo funciona?; en donde explicaremos como implementamos el corrector; III. Evaluación y IV. Conclusiones.

## **II. ¿Cómo funciona?**

Comenzamos por importar las librerias que utilizaremos


In [1]:
import re
from collections import Counter

Al realizar un corrector ortográfico, estamos intentando encontrar la corrección $c$, dentro de todos los posibles candidatos, de tal forma que maximize la probabilidad de que $c$ es la corrección correcta, dada la palabra original $w$.

$$argmax_{c \in candidates} P(c|w)$$

Por el teorema de Bayes, esta expresión es equivalente a

$$argmax_{c \in candidates} P(c) \dfrac{P(w|c)}{P(w)}$$

Como $P(w)$ es la misma para cualquier candidato $c$, podemos expresarlo de la siguiente manera:

$$argmax_{c \in candidates} P(c) P(w|c)$$

Esta expresión la podemos separar en cuatro partes:

### `1) Selection Mechanism`

---

Elegimos el candidato con mayor probabilidad combinada ($argmax$), esto lo vemos en la función `correccion`, con la palabra reservada `max` y el argumento `key`. 

La función `max` regresa el elemento más grande de un conjunto de elementos y `key` es una función en la que los elementos son pasados y cuyo valor de regreso es el utilziado para realizar la comparación.

```python
def correction(word):
    return max(candidates(word), key=P)
```

### `2) Candidate Model`

---

Definimos a la edición simple de una cadena de caracteres como la eliminación, sustitución o inserción de algún caracter en la cadena original. La  función edits1  dada una palabra regresa el conjunto de todas las cadenas de caracteres que se pueden obtener al realizar alguna de las tres operaciones previamente mencionadas en la palabra dada.

Por otro lado, la función edits2 obtiene una palabra, y regresa el conjunto de todas las cadenas que se pueden obtener al realizar dos operaciones en dicha palabra.

In [6]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmñnopqrstuvwxyzáéíóú'
    splits     = [(word[:i], word[i:])    for i in range(len(word)+1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in (set(letters)-set(R[0]))]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes+replaces+inserts)

In [ ]:
def edits2(word): 
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))-edits1(word)-set(word)




Otra forma de decidir que tan similares son dos palabras es utilizando algún tipo de métrica o distancia. Una métrica es la distancia de mínima edición, o la distancia de Levenshtein, la cual considera las operaciones previamente mencionadas. En este caso, se utilizará una versión de esta métrica que asigna un peso o valor de una unidad a cada operación realizada sobre la cadena de caracteres. Así, por ejemplo, la distancia entre "hola" y "ho", y entre "como" y "cosa" es de 2. Existe también una versión de esta misma métrica que asigna un peso de 2 a la operación de sustitución o reemplazo.

Ante esto, es claro que dada una palabra "word", el conjunto de cadenas de edits1(word) está contenido en el conjunto de cadenas que está a una distancia de 1 de dicha palabra.

Al considerar textos en español, nuestro conjunto de letras ("letters") posee ahora 32 símbolos, esto es, las 26 que comparte con el alfabeto inglés, más las cinco vocales con acento, mas el caracter "ñ". Así, con una cadena de caracteres de longitud $n$, podemos obtener $n$ cadenas al eliminar un simbolo de la cadena original, $31n$ al realizar reemplazos y $32(n+1)-n$ al insertar símbolos, esto es, existen en total $63n+32$ cadenas que están a una distancia igual a 1 de la palabra original.

Considerando el código, dada una cadena de longitud $n$ formamos $n+1$ "splits" (pares de la forma $(L,R)$). Para formar los "deletes", consideramos los $n$ splits $(L,R)$ tales que $R$ no es vacío, lo cual implica que hay $n$ cadenas que se pueden obtener eliminando un caracter de la palabra original. Por otro lado, para los replaces utilizamos todos los splits $(L,R)$ tales que $R$ es no vacío y todas las letras de nuestro alfabeto menos la eliminada, esto es, hay $31n$ cadenas que se pueden obtener a partir de un "replace". 

Finalmente, en inserts utilizamos todos los splits y todas las letras del alfabeto, por ende, se pueden obtener $32(n+1)$ cadenas al insertar un caracter en la cadena original. Sin embargo, debemos restar a este número $n$, pues en estas inserciones repetimos 2 $n$ palabras. Por ejemplo, con la cadena "hola" las cadenas "hhola", "hoola", "holla" y "holaa" se repiten dos veces. Esto se debe a que obtenemos la misma cadena al realizar la inserciones $\textbf{h}hola$ y $h\textbf{h}ola$. Así, la función edits1 devuelve un conjunto de cardinalidad $n+31n+32(n+1)-n=63n+32$ dada una palabra de longitud $n$.

Así, dada una palabra word de longitud $n$, como edits1(word) está contenido en el conjunto cadenas que están a una distancia igual a 1 de la cadena original, y ambos conjuntos son de cardinalidad $63n+32$ se concluye que en efecto estos dos conjuntos son iguales.

Con esto probado, es claro que el conjunto edits2(word) es igual al conjunto de cadenas que están a una distancia de Levenshtein igual a 2 de la misma palabra. Con la instrucción "-edits1(word)-set(word)" quitamos las cadenas que están a una distancia menor o igual a 1 de la palabra original.

Cabe aclarar que si utilizamos la otra versión de la distancia de Levenshtein (donde se asigna un peso de 2 en los reemplazos) la igualdad entre los conjuntos previamente descritos no se da, si no que el conjunto edits1(word) es el conjunto de palabras con distancia igual a 1 o 2, mientras que edits2(word) da el conjunto de cadenas con distancia 2, 3 o 4.

De forma similar, podemos definir la función edits3, la cual nos puede ayudar a encontrar todas las cadenas que están a una distancia de 3 unidades de la palabra original. Utilizando un método análogo al usado para definir la función edits2, edits3 se puede escribir como:

In [ ]:
def edits3(word): return set(e3 for e2 in edits2(word) for e3 in edits1(e2))

No se utilizará esta última función, pues dura un tiempo considerable en ejecutarse. 

Debido a que el número de cadenas que uno obtiene usualmente al utilizar edits1 o edits2, resulta conveniente restringirnos exclusivamente a las palabras que tenemos en nuestro vocabulario. Para esta tarea, definimos la función known:

In [ ]:
def known(words, vocab): 
    return set(w for w in words if w in vocab)

Cabe resaltar que en esta función, el parámetro "words" es un conjunto de palabras (o un vocabulario), mientras que en las funciones previas se utilizaba como parámetro "word" (una palabra).

### `3) Language Model`: $P(c)$

---

$P(c)$, es decir, la probabilidad que $c$ aparezca en nuestro texto. Para esto contamos el número de veces que cada tipo aparece en nuestro vocabulario.

Para esta tarea, utilizamos como *corpus* el libro de *Don Quijote de la Mancha* de *Miguel de Cervantes Saavedra* obtenido de [Project Gutenberg]("https://www.gutenberg.org/cache/epub/2000/pg2000.txt").

Para obtener nuestro vocabulario `WORDS` se utilizo un procesamiento sencillo de texto, en el que solo nos quedamos con caracteres alfanuméricos `re.findall(r'\w+', text.lower())` y los convertimos a minúscula `text.lower()`.

Una vez obtuvimos nuestro vocabulario, calculamos la frecuencia de cada tipo mediante la función `Counter`.



In [3]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('quijote.txt', encoding='utf-8').read()))

Para obtener la probabilidad $P(c)$, se creo una función `P(word, N)` que estima la probabilidad de cada palabra basada en su frecuencia. Donde `N` es la cantidad de palabras de nuestro corpus.

In [7]:
def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

In [8]:
print(f"Cardinalidad del vocabulario WORDS: {len(WORDS)}")
print(f"Cantidad de palabras del quijote: {sum(WORDS.values())}")
print(f"Palabras más comunes:")
for word in (WORDS.most_common(10)):
    print(word)
print(f"Probabilidad de 'que': {P('que')}")

Cardinalidad del vocabulario WORDS: 22942
Cantidad de palabras del quijote: 381226
Palabras más comunes:
('que', 20628)
('de', 18214)
('y', 18189)
('la', 10363)
('a', 9824)
('en', 8242)
('el', 8210)
('no', 6335)
('los', 4748)
('se', 4691)
Probabilidad de 'que': 0.0541096357541196


### `4) Error Model`: $P(w|c)$

---

## **III. Evaluación**

## **IV. Conclusiones**